In [2]:
# !pip install simpy
# !pip install rbfopt

In [29]:
# import simpy
import numpy as np
import rbfopt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import simpy
import pprint

# Material Flow Simulation Model 1

In [65]:
# Initialize input
# Define the manufacturing process stages
num_stages = 12
station = ['Lamination - Cores', 'Machining', 'Internal Circuitize', 'Op Test/Repair - Internal',
'Lamination - Composites','External Circuitize','Op Test/Repair - External','Drilling',
'Copper plating','Procoat', 'Sizing', 'EOL Test']

process_data = {
    'No. of Tools': [3, 1, 2, 2, 2, 3, 2, 31, 2, 4, 2, 1],  # Example data, replace with actual values
    'Base Rate': [94, 883, 94, 105.7, 156, 70, 105.7, 8.2, 90, 48.1, 157.4, 243],
    'Efficiency': [0.8, 0.98, 0.95, 0.95, 0.8, 0.9, 0.95, 0.81, 0.9, 0.8, 0.85, 0.98],
    'MTTF': [57, 72, 352, 1, 57, 351, 1, 232, 16, 40, 1, 200],
    'MTTR': [3, 3, 3, 0, 3, 3, 0, 9.6, 3, 4, 0, 1.6],
    'Lot Size': [300, 60, 120, 60, 120, 120, 60, 60, 60, 60, 60, 60],
    'Setup Time': [0.5, 0.25, 0.75, 0.2, 0.5, 0.33, 0.2, 0.6, 0, 0.33, 0.5, 0.1],
    'Non-bottleneck Time': [0, 0.16, 1.5, 0.16, 0.5, 2, 0.16, 0.16, 0.16, 2.1, 0.16, 0.16]  # Placeholder for non-bottleneck time
}

# Simulation parameters
total_panels = 3000  # Total panels to process

In [66]:
class MaterialFlowSimulator():
  def __init__(self, stages:list, lpl_parameters:dict, no_of_inputs:int):
    self.stages = stages
    self.lpl_parameters = lpl_parameters
    self.no_of_inputs = no_of_inputs
    self.num_stages = len(self.stages)
    self.capacities, self.cycle_times = self.compute_lpl_capacity()
    self.bottleneck = self.bottleneck()

  def compute_lpl_capacity(self):
    # Initialize variables to store results
    output_at_stages = []
    cycle_times = []
    capacities = []

    process_data = self.lpl_parameters
    for stage in range(self.num_stages):
      # Calculate capacity and cycle time
      availability = process_data['MTTF'][stage] / (process_data['MTTF'][stage] + process_data['MTTR'][stage])
      sum_parameter = process_data['Efficiency'][stage] * process_data['No. of Tools'][stage] * process_data['Base Rate'][stage] * availability
      process_time_per_batch = process_data['Lot Size'][stage] / sum_parameter
      process_time_per_ops = process_time_per_batch + process_data['Setup Time'][stage] + process_data['Non-bottleneck Time'][stage]

      capacity = (19.5 * process_data['Lot Size'][stage]) / process_time_per_ops
      cycle_time = process_time_per_ops / process_data['No. of Tools'][stage]

      capacities.append(capacity)
      cycle_times.append(cycle_time)

      # Test
      # print(self.stages[stage], capacity)

    return capacities, cycle_times

  def bottleneck(self):
    ''' Output a ascending dataframe based on capacity.
        Shows the bottleneck.
    '''
    bottleneck_df = pd.DataFrame({
        'stations': self.stages,
        'capacity': self.capacities,
        'cycle time': self.cycle_times

    })

    bottleneck = {
        'dataframe': bottleneck_df,
        'bottleneck_set': bottleneck_df.sort_values(by='capacity', ascending=True)
    }
    return bottleneck


  def simulatorA(self, bottleneck:str, param_grid_1:list, param_grid_2:list):
    ''' Simulate the possible capacities and cycle time
        using different non-bottleneck time and no. of tools.
    '''
    bottleneck_index = self.stages.index(bottleneck)
    process_data = self.lpl_parameters

    base_rate = process_data['Base Rate'][bottleneck_index]
    eff = process_data['Efficiency'][bottleneck_index]
    mttf = process_data['MTTF'][bottleneck_index]
    mttr = process_data['MTTR'][bottleneck_index]
    lot_size = process_data['Lot Size'][bottleneck_index]
    setup_time = process_data['Setup Time'][bottleneck_index]

    # capacity_df = pd.DataFrame(columns=['No. of Tools', 'Non-bottleneck time', 'Capacity', 'Cycle Time'])
    capacity_list = []
    for tools in param_grid_1:
      for nbtime in param_grid_2:
        avail = mttf / (mttf + mttr)
        batch_process_time = lot_size / (avail * base_rate * tools * eff)
        raw_process_time = setup_time + batch_process_time + nbtime

        capacity = (19.5 * lot_size) / raw_process_time
        cycle_time = raw_process_time / tools

        capacity_row = {'No. of Tools': tools,
                        'Non-bottleneck time': nbtime,
                        'Capacity': capacity,
                        'Cycle Time': cycle_time}
        # Test
        # print(capacity_row)
        capacity_list.append(capacity_row)
    capacity_df = pd.DataFrame.from_dict(capacity_list)
    # Test
    # print(capacity_df)


    # Visualize
    fig = px.scatter(capacity_df,
                  x='Capacity',
                  y='Cycle Time',
                  color='Non-bottleneck time',
                  size='No. of Tools').update_layout(template='plotly_white', title=bottleneck)

    return capacity_df, fig

  def simulate_production(self, processes,target_units):
    # dataset = self.bottleneck
    # dataset = dataset['dataframe']
    # dataset['units'] = 0
    # processes = dataset.to_dict('records')
    # print(processes)

    total_units = 0
    time_elapsed = 0

    while total_units < target_units:
      # Find the process with the minimum time to produce one unit
        min_time_process = min(processes, key=lambda x: x['cycle time'] / x['capacity'])

        # Calculate the time required to produce the next unit
        time_to_produce = min_time_process['cycle time'] / min_time_process['capacity']

        # Update the total units and time elapsed
        total_units += 1
        time_elapsed += time_to_produce

        # Update the units produced in the process
        for process in processes:
            units_produced = min(process['capacity'] * time_to_produce, target_units - total_units)
            process['units'] += units_produced

        # Check if the target number of units is achieved
        if total_units >= target_units:
            break
    # Set identifier for each process if the target was achieved or not
    for process in processes:
        process['achieved_target'] = 'Yes' if process['units'] >= target_units else 'No'
    return total_units, time_elapsed

In [67]:
simulation = MaterialFlowSimulator(station, process_data, total_panels)
simulation.bottleneck['bottleneck_set']

,stations,capacity,cycle time
9,Procoat,409.263461,0.714699
5,External Circuitize,787.786665,0.990116
2,Internal Circuitize,799.284373,1.463809
7,Drilling,1100.183741,0.034305
10,Sizing,1323.182143,0.442116
4,Lamination - Composites,1553.709677,0.753036
3,Op Test/Repair - Internal,1776.063728,0.329380
6,Op Test/Repair - External,1776.063728,0.329380
8,Copper plating,1950.602038,0.299907
11,EOL Test,2276.405188,0.513968


In [70]:
# Simulate production
processes = simulation.bottleneck['dataframe']
processes['units'] = 0
processes = processes.to_dict('records')

**Using this station parameters, how many units can the system achieve?**

In [72]:
# Define a process function for each station in the assembly line
def station_process(env, station_name, capacity, cycle_time, out):
    while True:
        # Process units as per the capacity and cycle time
        yield env.timeout(cycle_time)
        # print(f"{env.now:.2f}: Unit processed at {station_name}")
        out.append(1)  # Track the processed units

# Define the simulation environment
def run_simulation(stations_data, target_units, total_hours):
    env = simpy.Environment()
    units_processed = []  # List to track processed units

    # Create processes for each station in the assembly line
    for station in stations_data:
        env.process(station_process(env, station['stations'], station['capacity'], station['cycle time'], units_processed))

    # Run the simulation
    sim_duration = total_hours  # Change this to set the simulation time
    env.run(until=sim_duration)

    # Calculate units processed at the end of the line
    total_units_processed = len(units_processed)
    print(f"\nSimulation finished.\nTotal Units Processed: {total_units_processed}")
    print(f"Target Units: {target_units}\n")

# Input data for each station
stations_data = processes

# Input parameters for the simulation
target_units = 3000  # Change this to set the target units
total_working_hours = 19.5 # Change this to set the total working hours

# Run the simulation with the provided input data and parameters
run_simulation(stations_data, target_units, total_working_hours)


Simulation finished.
Total Units Processed: 324
Target Units: 1000



The total units processed is 986, this is significantly lower than the expected 1400 panels/days stated in the case study. This means, that the 3000 panels/days is unachievable. In order to properly tune the number of yields per day, we have to level it to its expected value before targeting the 3000 units.

Upon seeing the variables given in the data collection, we have observe that the capacity is highly affected by the no. of tools/machines used during processing. Increasing the no. of tools/machines will increase the capacity of the stations. On the other hand, cycle time is highly dependent to non-bottleneck time. Typical value of non-bottleneck time is 0.16 (9.6 minutes), this assumes that the station is placed properly place to its previous stations. Reducing the non-bottleneck time means the layout should be properly rearrange.

In order to determine either of the two variable are more effective to tune, we conducted a simulation of different values for number of tools and non-bottleneck time.

In [132]:
# Simulate procoat
tools_ = [4, 5, 6, 7, 8, 9, 10]
nbtime_ = [2.1, 0.5, 0.16, 0.05]

procoat, compare_viz = simulation.simulatorA('Procoat', tools_, nbtime_)
compare_viz.show()

In [75]:
procoat

,No. of Tools,Non-bottleneck time,Capacity,Cycle Time
0,4,2.10,409.263461,0.714699
1,4,0.50,929.460924,0.314699
2,4,0.16,1273.408155,0.229699
3,4,0.05,1446.597949,0.202199
4,5,2.10,421.920335,0.554607
5,5,0.50,997.412403,0.234607
6,5,0.16,1404.502234,0.166607
7,5,0.05,1618.178159,0.144607
8,6,2.10,430.802324,0.452644
9,6,0.50,1048.516014,0.185977


In [133]:
# Simulate Internal Circuitize
tools_ = [2, 3, 4, 5, 6, 7, 8]
nbtime_ = [2.1, 0.5, 0.16, 0.05]

IC, IC_compare = simulation.simulatorA('Internal Circuitize', tools_, nbtime_)
IC_compare.show()

In [77]:
IC

,No. of Tools,Non-bottleneck time,Capacity,Cycle Time
0,2,2.10,663.336970,1.763809
1,2,0.50,1213.932928,0.963809
2,2,0.16,1473.905399,0.793809
3,2,0.05,1583.628954,0.738809
4,3,2.10,708.715955,1.100582
5,3,0.50,1375.058401,0.567249
6,3,0.16,1718.382259,0.453915
7,3,0.05,1869.388983,0.417249
8,4,2.10,733.816195,0.797202
9,4,0.50,1472.800929,0.397202


In [134]:
# Simulate External Circuitize
tools_ = [2, 3, 4, 5, 6, 7, 8]
nbtime_ = [2.1, 0.5, 0.16, 0.05]

EC, EC_compare = simulation.simulatorA('External Circuitize', tools_, nbtime_)
EC_compare.show()

In [79]:
EC

,No. of Tools,Non-bottleneck time,Capacity,Cycle Time
0,2,2.10,690.159426,1.695260
1,2,0.50,1306.882216,0.895260
2,2,0.16,1613.213503,0.725260
3,2,0.05,1745.590012,0.670260
4,3,2.10,762.128765,1.023449
5,3,0.50,1591.460731,0.490116
6,3,0.16,2070.160194,0.376782
7,3,0.05,2293.336773,0.340116
8,4,2.10,804.051739,0.727565
9,4,0.50,1785.904433,0.327565


In general, we have established a conclusion that reducing the non-bottleneck time to 0.16 should be the priority. However, decreasing the non-bottleneck time does not guarantee the increase of capacity, so increasing the number of tools/machines should be the next step to improve the performance of the system.

The next step we proceeded is creating a parameter grid that contains the capacity and cycle time of procoat, internal circuitize, external circuitize with 0.16 non-bottleneck time. Using this parameter grid, we will determine which combination will yield the target of more than 1400 panels.

# Model 3: Validation

In [101]:
print("Redundant test cycle")
# Define a process function for each station in the assembly line
def station_process(env, station_name, capacity, cycle_time, out):
    while True:
        # Process units as per the capacity and cycle time
        yield env.timeout(cycle_time)
        # print(f"{env.now:.2f}: Unit processed at {station_name}")
        out.append(1)  # Track the processed units

# Define the simulation environment
def run_simulation(stations_data, target_units, total_hours):
    env = simpy.Environment()
    units_processed = []  # List to track processed units

    # Create processes for each station in the assembly line
    for station in stations_data:
        env.process(station_process(env, station['stations'], station['capacity'], station['cycle time'], units_processed))

    # Run the simulation
    sim_duration = total_hours  # Change this to set the simulation time
    env.run(until=sim_duration)

    # Calculate units processed at the end of the line
    total_units_processed = len(units_processed)
    print(f"\nSimulation finished.\nTotal Units Processed: {total_units_processed}")
    print(f"Target Units: {target_units}\n")

# Input data for each station
stations_data = processes

# Input parameters for the simulation
target_units = 1500  # Change this to set the target units
total_working_hours = 19.5 # Change this to set the total working hours

# Run the simulation with the provided input data and parameters
run_simulation(stations_data, target_units, total_working_hours)

Redundant test cycle

Simulation finished.
Total Units Processed: 1578
Target Units: 1500



### **Simulate Procoat only**

In [102]:
import simpy

# Define a process function for each station in the assembly line
def station_process(env, station_name, capacity, cycle_time, out):
    while True:
        # Process units as per the capacity and cycle time
        yield env.timeout(cycle_time)
        # print(f"{env.now:.2f}: Unit processed at {station_name}")
        out.append(1)  # Track the processed units

# Define the simulation environment
def run_simulation(stations_data, target_units, total_hours):
    env = simpy.Environment()
    units_processed = []  # List to track processed units

    # Create processes for each station in the assembly line
    for station in stations_data:
        env.process(station_process(env, station['stations'], station['capacity'], station['cycle time'], units_processed))

    # Run the simulation
    sim_duration = total_hours  # Change this to set the simulation time
    env.run(until=sim_duration)

    # Calculate units processed at the end of the line
    total_units_processed = len(units_processed)

    # Test
    print(f"\nSimulation finished.\nTotal Units Processed: {total_units_processed}")
    print(f"Target Units: {target_units}\n")
    return total_units_processed

# Input data for each station
stations_data = processes

# Input parameters for the simulation
target_units = 1500  # Change this to set the target units
total_working_hours = 19.5  # Change this to set the total working hours

for random_parameter in procoat_data:
    # Modify the capacity for Station 2
    stations_data[9]['capacity'] = random_parameter[0]  # Assuming Station 2 is at index 1 in the stations_data list
    stations_data[9]['cycle time'] = random_parameter[1]
    print(f"{stations_data[9]['stations']}: {random_parameter[0]}, Cycle time {random_parameter[1]}")
    run_simulation(stations_data, target_units, total_working_hours)

Procoat: 1507.998606607894, Cycle time 0.1293104643104643

Simulation finished.
Total Units Processed: 1434
Target Units: 1500

Procoat: 1591.7820617981838, Cycle time 0.10500360643217788

Simulation finished.
Total Units Processed: 1469
Target Units: 1500

Procoat: 1660.9949382406326, Cycle time 0.08804963617463618

Simulation finished.
Total Units Processed: 1505
Target Units: 1500

Procoat: 1719.1340739458492, Cycle time 0.07561946561946561

Simulation finished.
Total Units Processed: 1541
Target Units: 1500

Procoat: 1768.6602344511139, Cycle time 0.06615176715176715

Simulation finished.
Total Units Processed: 1578
Target Units: 1500



In [103]:
# Select best data
stations_data = processes
procoat_data = []
for index, procoats in procoat.iterrows():
  if procoats['Capacity'] >= 1500 and procoats['Non-bottleneck time'] == 0.16:
    procoat_data.append((procoats['Capacity'], procoats['Cycle Time']))

ic_data = []
for index, ic in IC.iterrows():
  if ic['Capacity'] >= 1500 and ic['Non-bottleneck time'] == 0.16:
    ic_data.append((ic['Capacity'], ic['Cycle Time']))

ec_data = []
for index, ec in EC.iterrows():
  if ec['Capacity'] >= 1500 and ec['Non-bottleneck time'] == 0.16:
    ec_data.append((ec['Capacity'], ec['Cycle Time']))

In [45]:
# pprint.pprint(procoat_data)
# pprint.pprint(ic_data)
# pprint.pprint(ec_data)

In [104]:
multi_simulation_list = []

for procoat_parameter in procoat_data:
  stations_data[9]['capacity'] = procoat_parameter[0]
  stations_data[9]['cycle time'] = procoat_parameter[1]

  for internal_circuitize_parameter in ic_data:
    stations_data[2]['capacity'] = internal_circuitize_parameter[0]
    stations_data[2]['cycle time'] = internal_circuitize_parameter[1]

    for external_circuitize_parameter in ec_data:
      stations_data[5]['capacity'] = external_circuitize_parameter[0]
      stations_data[5]['cycle time'] = external_circuitize_parameter[1]
      units_yield = run_simulation(stations_data, target_units, total_working_hours)
      multi_simulation_row = {
          'pr_capacity': procoat_parameter[0],
          'pr_cycle_time': procoat_parameter[1],
          'ic_capacity': internal_circuitize_parameter[0],
          'ic_cycle_time': internal_circuitize_parameter[1],
          'ec_capacity': external_circuitize_parameter[0],
          'ec_cycle_time': external_circuitize_parameter[1],
          'units_yield': units_yield
      }
      multi_simulation_list.append(multi_simulation_row)
multi_simulation_df = pd.DataFrame.from_dict(multi_simulation_list)

# locate the yield >= 1500
multi_simulation_df.loc[multi_simulation_df['units_yield'] >= 1400]


Simulation finished.
Total Units Processed: 1145
Target Units: 1500


Simulation finished.
Total Units Processed: 1170
Target Units: 1500


Simulation finished.
Total Units Processed: 1199
Target Units: 1500


Simulation finished.
Total Units Processed: 1230
Target Units: 1500


Simulation finished.
Total Units Processed: 1263
Target Units: 1500


Simulation finished.
Total Units Processed: 1297
Target Units: 1500


Simulation finished.
Total Units Processed: 1332
Target Units: 1500


Simulation finished.
Total Units Processed: 1165
Target Units: 1500


Simulation finished.
Total Units Processed: 1190
Target Units: 1500


Simulation finished.
Total Units Processed: 1219
Target Units: 1500


Simulation finished.
Total Units Processed: 1250
Target Units: 1500


Simulation finished.
Total Units Processed: 1283
Target Units: 1500


Simulation finished.
Total Units Processed: 1317
Target Units: 1500


Simulation finished.
Total Units Processed: 1352
Target Units: 1500


Simulation finished

,pr_capacity,pr_cycle_time,ic_capacity,ic_cycle_time,ec_capacity,ec_cycle_time,units_yield
34,1507.998607,0.129310,2120.323105,0.157658,3204.907661,0.091266,1413
41,1507.998607,0.129310,2167.861573,0.134926,3204.907661,0.091266,1434
62,1591.782062,0.105004,1981.291961,0.236210,3204.907661,0.091266,1407
69,1591.782062,0.105004,2060.089553,0.189312,3204.907661,0.091266,1428
75,1591.782062,0.105004,2120.323105,0.157658,3061.086092,0.109205,1413
...,...,...,...,...,...,...,...
205,1768.660234,0.066152,2167.861573,0.134926,2411.723499,0.242565,1445
206,1768.660234,0.066152,2167.861573,0.134926,2676.707341,0.174842,1476
207,1768.660234,0.066152,2167.861573,0.134926,2888.269681,0.135029,1509
208,1768.660234,0.066152,2167.861573,0.134926,3061.086092,0.109205,1543


In [105]:
multi_simulation_df_1400 = multi_simulation_df.loc[multi_simulation_df['units_yield'] >= 1400]
multi_simulation_df_1400

,pr_capacity,pr_cycle_time,ic_capacity,ic_cycle_time,ec_capacity,ec_cycle_time,units_yield
34,1507.998607,0.129310,2120.323105,0.157658,3204.907661,0.091266,1413
41,1507.998607,0.129310,2167.861573,0.134926,3204.907661,0.091266,1434
62,1591.782062,0.105004,1981.291961,0.236210,3204.907661,0.091266,1407
69,1591.782062,0.105004,2060.089553,0.189312,3204.907661,0.091266,1428
75,1591.782062,0.105004,2120.323105,0.157658,3061.086092,0.109205,1413
...,...,...,...,...,...,...,...
205,1768.660234,0.066152,2167.861573,0.134926,2411.723499,0.242565,1445
206,1768.660234,0.066152,2167.861573,0.134926,2676.707341,0.174842,1476
207,1768.660234,0.066152,2167.861573,0.134926,2888.269681,0.135029,1509
208,1768.660234,0.066152,2167.861573,0.134926,3061.086092,0.109205,1543


In [140]:
## Rename columns before joining
procoat.rename(columns={'Capacity': 'pr_capacity', 'Cycle Time': 'pr_cycle_time', 'No. of Tools': 'pr_tools', 'Non-bottleneck time': 'pr_nbt'}, inplace=True)
IC.rename(columns={'Capacity': 'ic_capacity', 'Cycle Time': 'ic_cycle_time', 'No. of Tools': 'ic_tools', 'Non-bottleneck time': 'ic_nbt'}, inplace=True)
EC.rename(columns={'Capacity': 'ec_capacity', 'Cycle Time': 'ec_cycle_time', 'No. of Tools': 'ec_tools', 'Non-bottleneck time': 'ec_nbt'}, inplace=True)

In [176]:
# Merge DataFrames based on matching pr_capacity and pr_cycle_time columns
merged_procoat = pd.merge(multi_simulation_df_1400, procoat, on=['pr_capacity', 'pr_cycle_time'], how='inner')
merged_IC = pd.merge(merged_procoat, IC, on=['ic_capacity', 'ic_cycle_time'], how='inner')
merged_final = pd.merge(merged_IC, EC, on=['ec_capacity', 'ec_cycle_time'], how='inner')

# Display the resulting merged DataFrame
merged_final

,pr_capacity,pr_cycle_time,ic_capacity,ic_cycle_time,ec_capacity,ec_cycle_time,units_yield,pr_tools,pr_nbt,ic_tools,ic_nbt,ec_tools,ec_nbt
0,1507.998607,0.129310,2120.323105,0.157658,3204.907661,0.091266,1413,6,0.16,7,0.16,8,0.16
1,1591.782062,0.105004,2120.323105,0.157658,3204.907661,0.091266,1448,7,0.16,7,0.16,8,0.16
2,1660.994938,0.088050,2120.323105,0.157658,3204.907661,0.091266,1484,8,0.16,7,0.16,8,0.16
3,1719.134074,0.075619,2120.323105,0.157658,3204.907661,0.091266,1520,9,0.16,7,0.16,8,0.16
4,1768.660234,0.066152,2120.323105,0.157658,3204.907661,0.091266,1557,10,0.16,7,0.16,8,0.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,1768.660234,0.066152,2120.323105,0.157658,2411.723499,0.242565,1424,10,0.16,7,0.16,4,0.16
63,1719.134074,0.075619,2167.861573,0.134926,2411.723499,0.242565,1408,9,0.16,8,0.16,4,0.16
64,1768.660234,0.066152,2167.861573,0.134926,2411.723499,0.242565,1445,10,0.16,8,0.16,4,0.16
65,1768.660234,0.066152,2060.089553,0.189312,2411.723499,0.242565,1404,10,0.16,6,0.16,4,0.16


# **Summarizing the yield**

In [185]:
pr_capacity_1507 = merged_final.loc[merged_final['pr_capacity'] <= 1508]
pr_capacity_1507

,pr_capacity,pr_cycle_time,ic_capacity,ic_cycle_time,ec_capacity,ec_cycle_time,units_yield,pr_tools,pr_nbt,ic_tools,ic_nbt,ec_tools,ec_nbt
0,1507.998607,0.12931,2120.323105,0.157658,3204.907661,0.091266,1413,6,0.16,7,0.16,8,0.16
5,1507.998607,0.12931,2167.861573,0.134926,3204.907661,0.091266,1434,6,0.16,8,0.16,8,0.16


In [207]:
pd.DataFrame(pr_capacity_1507.groupby(['pr_tools','ec_tools', 'ic_tools', ])['units_yield'].sum())

units_yield
pr_tools ec_tools ic_tools             
6        8        7                1413
                  8                1434

In [208]:
pr_capacity_1591 = merged_final[(merged_final['pr_capacity'] >= 1591) & (merged_final['pr_capacity'] <= 1592)]
pd.DataFrame(pr_capacity_1591.groupby(['pr_tools', 'ec_tools', 'ic_tools',])['units_yield'].sum())

units_yield
pr_tools ec_tools ic_tools             
7        6        8                1400
         7        7                1413
                  8                1434
         8        5                1407
                  6                1428
                  7                1448
                  8                1469

In [220]:
pr_capacity_1660 = merged_final[(merged_final['pr_capacity'] >= 1660) & (merged_final['pr_capacity'] <= 1661)]
pd.DataFrame(pr_capacity_1660.groupby(['pr_tools', 'ec_tools', 'ic_tools',])['units_yield'].sum())

units_yield
pr_tools ec_tools ic_tools             
8        5        8                1403
         6        7                1415
                  8                1436
         7        5                1408
                  6                1429
                  7                1449
                  8                1470
         8        3                1403
                  4                1423
                  5                1443
                  6                1464
                  7                1484
                  8                1505

In [219]:
pr_capacity_1719 = merged_final[(merged_final['pr_capacity'] >= 1719) & (merged_final['pr_capacity'] <= 1722)]
pd.DataFrame(pr_capacity_1719.groupby(['pr_tools', 'ec_tools', 'ic_tools',])['units_yield'].sum()).sort_values(by='units_yield')

units_yield
pr_tools ec_tools ic_tools             
9        7        3                1404
         4        8                1408
         6        5                1410
         5        7                1418
         7        4                1424
         6        6                1431
         5        8                1439
         8        3                1439
         7        5                1444
         6        7                1451
         8        4                1459
         7        6                1465
         6        8                1472
         8        5                1479
         7        7                1485
         8        6                1500
         7        8                1506
         8        7                1520
                  8                1541

In [221]:
pr_capacity_1770 = merged_final[(merged_final['pr_capacity'] >= 1760) & (merged_final['pr_capacity'] <= 1800)]
pd.DataFrame(pr_capacity_1770.groupby(['pr_tools', 'ec_tools', 'ic_tools',])['units_yield'].sum())

units_yield
pr_tools ec_tools ic_tools             
10       3        8                1416
         4        6                1404
                  7                1424
                  8                1445
         5        5                1414
                  6                1435
                  7                1455
                  8                1476
         6        3                1407
                  4                1427
                  5                1447
                  6                1468
                  7                1488
                  8                1509
         7        3                1441
                  4                1461
                  5                1481
                  6                1502
                  7                1522
                  8                1543
         8        3                1476
                  4                1496
                  5                1516
                  6                1537
                  7                1557
                  8                1578